In [37]:
# Imports
import pandas as pd
import numpy as np
import datetime
from datetime import time as dtime
from datetime import timedelta
from datetime import date
import matplotlib as plt
from itertools import product
import openpyxl

# importing historical data for backtest
def get1minBars(csvName):
    
    priceData1minBars_df = pd.read_csv(csvName, usecols=['date', 'open', 'high', 'low', 'close', 'volume'])
    
    # Parse datetime column and divide in 2 columns
    # SPY_Data['datetime'] = pd.to_datetime(SPY_Data['date'].str.strip(), format='%Y%m%d %H:%M:%S')
    priceData1minBars_df.rename(columns={
        'date': 'datetime'
    }, inplace=True)
    
    priceData1minBars_df['datetime'] = pd.to_datetime(priceData1minBars_df['datetime'])
    
    priceData1minBars_df['Date'] = priceData1minBars_df['datetime'].dt.date
    priceData1minBars_df['Time'] = priceData1minBars_df['datetime'].dt.time
    
    # sorting the data by the time column to make sure it is consistent
    priceData1minBars_df.sort_values('datetime', inplace=True)
    
    # Drop original 'date' column
    # SPY_Data.drop(columns=['date'], inplace=True)
    
    # Filter for regular trading hours using datetime column
    # priceData1minBars_df = priceData1minBars_df[priceData1minBars_df['datetime'].dt.time >= pd.to_datetime("09:30").time()]
    # priceData1minBars_df = priceData1minBars_df[priceData1minBars_df['datetime'].dt.time <= pd.to_datetime("16:00").time()]
    
    # Making sure the order of the columns stays the same 
    priceData1minBars_df = priceData1minBars_df[['Date', 'Time', 'open', 'high', 'low', 'close', 'volume']]
    
    return priceData1minBars_df
    
    
NQ_Data = get1minBars('NQ_1min_master.csv')
# QQQ_Data = QQQ_Data[472980:483000]
# NQ_Data = NQ_Data[int(len(NQ_Data)/2):]
NQ_Data

,Date,Time,open,high,low,close,volume
0,2023-11-01,22:00:00,14761.25,14763.50,14760.00,14760.00,175
1,2023-11-01,22:01:00,14760.50,14764.00,14760.50,14762.00,156
2,2023-11-01,22:02:00,14762.25,14762.25,14760.50,14761.25,48
3,2023-11-01,22:03:00,14761.25,14762.00,14760.75,14761.00,30
4,2023-11-01,22:04:00,14761.75,14763.00,14761.00,14762.25,52
...,...,...,...,...,...,...,...
706568,2025-10-31,20:55:00,25988.50,25994.25,25987.75,25993.00,138
706569,2025-10-31,20:56:00,25993.50,25993.50,25988.25,25988.50,57
706570,2025-10-31,20:57:00,25989.00,25990.00,25985.00,25986.50,45
706571,2025-10-31,20:58:00,25984.75,25986.50,25981.50,25983.50,79


In [6]:
import pandas as pd
import numpy as np

# -------------------- Indicators --------------------
def smma(series: pd.Series, period: int) -> pd.Series:
    """Wilder-style Smoothed Moving Average (SMMA)."""
    s = series.astype(float).copy()
    out = pd.Series(index=s.index, dtype=float)
    alpha = 1.0 / period
    prev = np.nan
    for i, val in enumerate(s.values):
        if i == 0 or np.isnan(prev):
            prev = val
        else:
            prev = prev + alpha * (val - prev)
        out.iloc[i] = prev
    return out

def compute_alligator(close: pd.Series) -> tuple[pd.Series, pd.Series, pd.Series]:
    """
    Williams Alligator defaults:
      Jaw   = SMMA(13) shifted +8 bars
      Teeth = SMMA(8)  shifted +5 bars
      Lips  = SMMA(5)  shifted +3 bars
    """
    jaw   = smma(close, 13).shift(8)
    teeth = smma(close, 8).shift(5)
    lips  = smma(close, 5).shift(3)
    return jaw, teeth, lips

def ema(series: pd.Series, period: int) -> pd.Series:
    return series.astype(float).ewm(span=period, adjust=False).mean()

# -------------------- Normalizer for your schema --------------------
def normalize_from_date_time(df: pd.DataFrame, tz: str ) -> pd.DataFrame:
    """
    Accepts a DataFrame with columns:
      ['Date', 'Time', 'open', 'high', 'low', 'close', 'volume']
    where Date is datetime.date and Time is datetime.time (or strings).
    Produces a sorted DataFrame with a combined 'datetime' column.
    """
    out = df.copy()
    # Robust to weird dtypes: cast to string and combine
    out["datetime"] = pd.to_datetime(out["Date"].astype(str) + " " + out["Time"].astype(str))

    # Ensure numeric OHLCV
    for c in ["open","high","low","close","volume"]:
        out[c] = pd.to_numeric(out[c], errors="coerce")
    out = out.dropna(subset=["open","high","low","close"]).sort_values("datetime").reset_index(drop=True)

    # Optional timezone handling (usually not needed for pure intraday backtest)
    if tz:
        if out["datetime"].dt.tz is None:
            out["datetime"] = out["datetime"].dt.tz_localize(tz)
        else:
            out["datetime"] = out["datetime"].dt.tz_convert(tz)

    return out

# -------------------- Backtest (no CLI, runs on your DataFrame) --------------------
def backtest_alligator_on_df(
    df: pd.DataFrame,
    tick_size: float = 0.25,
    point_value: float = 20.0,     # NQ = $20/point (use 2.0 for MNQ micro, 50.0 for ES)
    stop_pts: float = 2.0,
    target_pts: float = 9.0,
    slippage_ticks: float = 1.0,
    session_close: str = "16:00:00",
    flat_overnight: bool = True
) -> tuple[pd.DataFrame, pd.Series]:
    """
    Expects df with columns: ['Date','Time','open','high','low','close','volume'].
    Uses Alligator + EMA(200). Entries fill next-bar-open. Stops/targets are fixed.
    """
    data = normalize_from_date_time(df, tz=None)

    # Indicators
    data["ema200"] = ema(data["close"], 200)
    jaw, teeth, lips = compute_alligator(data["close"])
    data["jaw"], data["teeth"], data["lips"] = jaw, teeth, lips

    in_pos = False
    side = 0       # +1 long, -1 short
    entry_px = np.nan
    entry_time = None
    stop_px = np.nan
    target_px = np.nan

    trades = []

    for i in range(1, len(data) - 1):
        row_prev = data.iloc[i - 1]
        row      = data.iloc[i]
        nxt      = data.iloc[i + 1]

        # Optional day flat
        if flat_overnight and row["datetime"].strftime("%H:%M:%S") >= session_close:
            if in_pos:
                exit_px = round(row["close"] / tick_size) * tick_size
                pnl_pts = (exit_px - entry_px) * side
                pnl_usd = pnl_pts * point_value
                trades.append({
                    "entry_time": entry_time,
                    "entry_px": entry_px,
                    "side": side,
                    "exit_time": row["datetime"],
                    "exit_px": exit_px,
                    "reason": "eod_flat",
                    "pnl_pts": pnl_pts,
                    "pnl_usd": pnl_usd
                })
                in_pos = False
                side = 0
            continue

        # Manage open position first
        if in_pos:
            if side == 1:
                if row["low"] <= stop_px:
                    exit_px = round(stop_px / tick_size) * tick_size
                    reason = "stop"
                elif row["high"] >= target_px:
                    exit_px = round(target_px / tick_size) * tick_size
                    reason = "target"
                else:
                    exit_px = None
            else:
                if row["high"] >= stop_px:
                    exit_px = round(stop_px / tick_size) * tick_size
                    reason = "stop"
                elif row["low"] <= target_px:
                    exit_px = round(target_px / tick_size) * tick_size
                    reason = "target"
                else:
                    exit_px = None

            if exit_px is not None:
                pnl_pts = (exit_px - entry_px) * side
                pnl_usd = pnl_pts * point_value
                trades.append({
                    "entry_time": entry_time,
                    "entry_px": entry_px,
                    "side": side,
                    "exit_time": row["datetime"],
                    "exit_px": exit_px,
                    "reason": reason,
                    "pnl_pts": pnl_pts,
                    "pnl_usd": pnl_usd
                })
                in_pos = False
                side = 0
                continue

        # Entries (when flat)
        if not in_pos:
            bias_long  = row["close"] > row["ema200"]
            bias_short = row["close"] < row["ema200"]
            mouth_up   = (row["lips"] > row["teeth"]) and (row["teeth"] > row["jaw"])
            mouth_dn   = (row["lips"] < row["teeth"]) and (row["teeth"] < row["jaw"])
            crossed_up = (row_prev["close"] <= row_prev["teeth"]) and (row["close"] > row["teeth"])
            crossed_dn = (row_prev["close"] >= row_prev["teeth"]) and (row["close"] < row["teeth"])

            if bias_long and mouth_up and crossed_up:
                raw = float(nxt["open"])
                entry_px = round((raw + slippage_ticks * tick_size) / tick_size) * tick_size
                side = 1
                entry_time = nxt["datetime"]
                stop_px = entry_px - stop_pts
                target_px = entry_px + target_pts
                in_pos = True
                continue

            if bias_short and mouth_dn and crossed_dn:
                raw = float(nxt["open"])
                entry_px = round((raw - slippage_ticks * tick_size) / tick_size) * tick_size
                side = -1
                entry_time = nxt["datetime"]
                stop_px = entry_px + stop_pts
                target_px = entry_px - target_pts
                in_pos = True
                continue

    trades_df = pd.DataFrame(trades)
    if trades_df.empty:
        summary = pd.Series({
            "n_trades": 0,
            "win_rate": np.nan,
            "total_pnl_pts": 0.0,
            "total_pnl_usd": 0.0,
            "avg_pnl_pts": np.nan,
            "max_drawdown_usd": 0.0
        })
        return trades_df, summary

    # Simple drawdown on cumulative USD PnL
    trades_df["cum_pnl_usd"] = trades_df["pnl_usd"].cumsum()
    roll_max = trades_df["cum_pnl_usd"].cummax()
    dd = trades_df["cum_pnl_usd"] - roll_max
    max_dd = float(dd.min()) if len(dd) else 0.0

    summary = pd.Series({
        "n_trades": int(len(trades_df)),
        "win_rate": float((trades_df["pnl_usd"] > 0).mean()),
        "total_pnl_pts": float(trades_df["pnl_pts"].sum()),
        "total_pnl_usd": float(trades_df["pnl_usd"].sum()),
        "avg_pnl_pts": float(trades_df["pnl_pts"].mean()),
        "max_drawdown_usd": max_dd
    })
    return trades_df, summary

# -------------------- Run directly on your NQ_Data --------------------
# Example (uses your NQ_Data DataFrame already created in your code above):
trades, summary = backtest_alligator_on_df(
    NQ_Data,
    tick_size=0.25,
    point_value=20.0,      # NQ = $20/point (use 2.0 for MNQ)
    stop_pts=2.0,
    target_pts=9.0,
    slippage_ticks=1.0,
    session_close="16:00:00",
    flat_overnight=True
)

print("=== SUMMARY ===")
print(summary)
print("\nFirst few trades:")
print(trades.head())

# If you want to save:
# trades.to_csv("NQ_alligator_trades.csv", index=False)
# summary.to_csv("NQ_alligator_summary.csv", header=False)


=== SUMMARY ===
n_trades             4731.000000
win_rate                0.155781
total_pnl_pts       -1355.000000
total_pnl_usd      -27100.000000
avg_pnl_pts            -0.286409
max_drawdown_usd   -27820.000000
dtype: float64

First few trades:
           entry_time  entry_px  side           exit_time   exit_px reason  \
0 2023-11-02 00:07:00  14787.25     1 2023-11-02 00:12:00  14785.25   stop   
1 2023-11-02 01:49:00  14814.00     1 2023-11-02 01:49:00  14812.00   stop   
2 2023-11-02 02:24:00  14808.75     1 2023-11-02 02:25:00  14806.75   stop   
3 2023-11-02 02:52:00  14809.50     1 2023-11-02 02:52:00  14807.50   stop   
4 2023-11-02 05:04:00  14803.00    -1 2023-11-02 05:09:00  14805.00   stop   

   pnl_pts  pnl_usd  cum_pnl_usd  
0     -2.0    -40.0        -40.0  
1     -2.0    -40.0        -80.0  
2     -2.0    -40.0       -120.0  
3     -2.0    -40.0       -160.0  
4     -2.0    -40.0       -200.0  


In [38]:
import pandas as pd
import numpy as np

# ---------- indicators ----------
def smma(x: pd.Series, n: int) -> pd.Series:
    s = x.astype(float).copy()
    out = pd.Series(index=s.index, dtype=float)
    alpha = 1.0 / n
    prev = np.nan
    for i, v in enumerate(s.values):
        if i == 0 or np.isnan(prev):
            prev = v
        else:
            prev = prev + alpha * (v - prev)
        out.iat[i] = prev
    return out

def ema(x: pd.Series, n: int) -> pd.Series:
    return x.astype(float).ewm(span=n, adjust=False).mean()

def true_range(df: pd.DataFrame) -> pd.Series:
    prev_close = df["close"].shift(1)
    tr1 = df["high"] - df["low"]
    tr2 = (df["high"] - prev_close).abs()
    tr3 = (df["low"]  - prev_close).abs()
    return pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)

def atr(df: pd.DataFrame, n: int = 14) -> pd.Series:
    return smma(true_range(df), n)

# ---------- prep ----------
def normalize_from_date_time(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out["datetime"] = pd.to_datetime(out["Date"].astype(str) + " " + out["Time"].astype(str))
    for c in ["open","high","low","close","volume"]:
        out[c] = pd.to_numeric(out[c], errors="coerce")
    out = out.dropna(subset=["open","high","low","close"]).sort_values("datetime").reset_index(drop=True)
    return out

# ---------- backtest v2 (unshifted alligator + ATR exits + time window) ----------
def backtest_alligator_v2(
    raw_df: pd.DataFrame,
    tick_size=0.25,
    point_value=20.0,           # NQ = $20/pt (MNQ ~2.0)
    atr_len=14,
    stop_mult=0.75,             # stop = 0.75*ATR
    target_mult=2.0,            # target = 2.0*ATR
    session_start="09:45:00",   # skip open churn
    session_end="15:55:00"      # skip last 5 min
):
    df = normalize_from_date_time(raw_df)

    # indicators
    df["ema200"] = ema(df["close"], 200)
    # UNshifted alligator (use for signals; shift only for plotting)
    df["jaw"]   = smma(df["close"], 13)   # no .shift()
    df["teeth"] = smma(df["close"], 8)
    df["lips"]  = smma(df["close"], 5)
    df["atr"]   = atr(df, atr_len)

    # simple trend slope filter
    df["ema_slope"] = df["ema200"] - df["ema200"].shift(10)

    # intraday filter
    t = df["datetime"].dt.time
    df = df[(t >= pd.to_datetime(session_start).time()) & (t <= pd.to_datetime(session_end).time())].copy()
    df.reset_index(drop=True, inplace=True)
    if len(df) < 50:
        return pd.DataFrame(), pd.Series({"n_trades":0})

    trades = []
    in_pos = False
    side   = 0
    entry_px = np.nan
    entry_time = None
    stop_px = np.nan
    tgt_px  = np.nan

    for i in range(1, len(df)-1):
        prev = df.iloc[i-1]
        row  = df.iloc[i]
        nxt  = df.iloc[i+1]

        # manage open
        if in_pos:
            if side == 1:
                if row["low"] <= stop_px:
                    exit_px, reason = stop_px, "stop"
                elif row["high"] >= tgt_px:
                    exit_px, reason = tgt_px, "target"
                else:
                    exit_px = None
            else:
                if row["high"] >= stop_px:
                    exit_px, reason = stop_px, "stop"
                elif row["low"] <= tgt_px:
                    exit_px, reason = tgt_px, "target"
                else:
                    exit_px = None

            if exit_px is not None:
                # round to tick
                exit_px = round(exit_px / tick_size) * tick_size
                pnl_pts = (exit_px - entry_px) * side
                pnl_usd = pnl_pts * point_value
                trades.append({
                    "entry_time": entry_time, "entry_px": entry_px, "side": side,
                    "exit_time": row["datetime"], "exit_px": exit_px, "reason": reason,
                    "pnl_pts": pnl_pts, "pnl_usd": pnl_usd
                })
                in_pos = False
                side = 0
                continue

        # entries only if flat
        if not in_pos:
            bias_long  = (row["close"] > row["ema200"]) and (row["ema_slope"] > 0)
            bias_short = (row["close"] < row["ema200"]) and (row["ema_slope"] < 0)

            # "mouth open" strength thresholds to avoid chop
            mouth_up = (row["lips"] > row["teeth"]) and (row["teeth"] > row["jaw"]) \
                       and ( (row["lips"] - row["teeth"]) > 0.15*row["atr"] ) \
                       and ( (row["teeth"] - row["jaw"])  > 0.15*row["atr"] )

            mouth_dn = (row["lips"] < row["teeth"]) and (row["teeth"] < row["jaw"]) \
                       and ( (row["teeth"] - row["lips"]) > 0.15*row["atr"] ) \
                       and ( (row["jaw"]  - row["teeth"]) > 0.15*row["atr"] )

            crossed_up = (prev["close"] <= prev["teeth"]) and (row["close"] > row["teeth"])
            crossed_dn = (prev["close"] >= prev["teeth"]) and (row["close"] < row["teeth"])

            if bias_long and mouth_up and crossed_up and row["atr"] > 0:
                raw = float(nxt["open"])
                entry_px = round((raw + 2*tick_size) / tick_size) * tick_size  # 2 tick slippage
                side = 1
                entry_time = nxt["datetime"]
                stop_px = entry_px - stop_mult*row["atr"]
                tgt_px  = entry_px + target_mult*row["atr"]
                in_pos = True
                continue

            if bias_short and mouth_dn and crossed_dn and row["atr"] > 0:
                raw = float(nxt["open"])
                entry_px = round((raw - 2*tick_size) / tick_size) * tick_size
                side = -1
                entry_time = nxt["datetime"]
                stop_px = entry_px + stop_mult*row["atr"]
                tgt_px  = entry_px - target_mult*row["atr"]
                in_pos = True
                continue

    trades_df = pd.DataFrame(trades)
    if trades_df.empty:
        summary = pd.Series({
            "n_trades": 0, "win_rate": np.nan,
            "total_pnl_pts": 0.0, "total_pnl_usd": 0.0,
            "avg_pnl_pts": np.nan, "max_drawdown_usd": 0.0
        })
        return trades_df, summary

    # stats
    trades_df["cum_pnl_usd"] = trades_df["pnl_usd"].cumsum()
    roll_max = trades_df["cum_pnl_usd"].cummax()
    dd = trades_df["cum_pnl_usd"] - roll_max
    summary = pd.Series({
        "n_trades": int(len(trades_df)),
        "win_rate": float((trades_df["pnl_usd"] > 0).mean()),
        "total_pnl_pts": float(trades_df["pnl_pts"].sum()),
        "total_pnl_usd": float(trades_df["pnl_usd"].sum()),
        "avg_pnl_pts": float(trades_df["pnl_pts"].mean()),
        "max_drawdown_usd": float(dd.min())
    })
    return trades_df, summary

# --- run on your NQ_Data ---
trades2, summary2 = backtest_alligator_v2(
    NQ_Data,
    tick_size=0.25, point_value=20.0,
    atr_len=14, stop_mult=0.75, target_mult=2.0,
    session_start="09:45:00", session_end="15:55:00"
)

print("=== SUMMARY v2 (unshifted + ATR + time-filter) ===")
print(summary2)
print("\nSample trades:")
trades2


=== SUMMARY v2 (unshifted + ATR + time-filter) ===
n_trades             2918.000000
win_rate                0.252570
total_pnl_pts        1356.750000
total_pnl_usd       27135.000000
avg_pnl_pts             0.464959
max_drawdown_usd    -6150.000000
dtype: float64

Sample trades:


,entry_time,entry_px,side,exit_time,exit_px,reason,pnl_pts,pnl_usd,cum_pnl_usd
0,2023-11-02 10:23:00,14855.25,1,2023-11-02 10:25:00,14852.50,stop,-2.75,-55.0,-55.0
1,2023-11-02 10:50:00,14859.00,1,2023-11-02 10:53:00,14866.00,target,7.00,140.0,85.0
2,2023-11-02 11:54:00,14879.00,1,2023-11-02 12:00:00,14875.75,stop,-3.25,-65.0,20.0
3,2023-11-02 13:03:00,14945.75,1,2023-11-02 13:03:00,14941.25,stop,-4.50,-90.0,-70.0
4,2023-11-02 15:03:00,14954.00,1,2023-11-02 15:14:00,14947.25,stop,-6.75,-135.0,-205.0
...,...,...,...,...,...,...,...,...,...
2913,2025-10-31 10:06:00,26179.25,-1,2025-10-31 10:12:00,26182.75,stop,-3.50,-70.0,26405.0
2914,2025-10-31 10:26:00,26166.50,-1,2025-10-31 10:29:00,26170.25,stop,-3.75,-75.0,26330.0
2915,2025-10-31 13:16:00,26206.25,-1,2025-10-31 13:30:00,26189.50,target,16.75,335.0,26665.0
2916,2025-10-31 14:08:00,26097.00,-1,2025-10-31 14:08:00,26112.50,stop,-15.50,-310.0,26355.0


In [19]:

def trade_stats(df, pnl_col='returns', r_col=None, equity_start=0.0):
    """
    df must have a PnL column (default: 'returns').
    If you also have an R-multiple column, pass its name via r_col.
    Returns (stats_dict, equity_series, drawdown_series).
    """
    # sort (optional, helps equity curve look right)
    if {'Date','timeExit'}.issubset(df.columns):
        df = df.sort_values(['Date','timeExit'])
    s = pd.Series(df[pnl_col].astype(float).values)
    n = int(s.size)
    if n == 0:
        return {}, pd.Series(dtype=float), pd.Series(dtype=float)

    wins   = s[s > 0]
    losses = s[s < 0]

    gross_profit = float(wins.sum())
    gross_loss   = float(-losses.sum())  # positive number

    win_rate  = float(len(wins) / (len(wins) + len(losses)))
    loss_rate = float(len(losses) / (len(wins) + len(losses)))

    avg_win  = float(wins.mean())   if len(wins)   else 0.0
    avg_loss = float(losses.mean()) if len(losses) else 0.0  # negative
    payoff   = (avg_win / abs(avg_loss)) if len(wins) and len(losses) else np.inf

    profit_factor = (gross_profit / gross_loss) if gross_loss > 0 else np.inf
    expectancy    = win_rate * avg_win + loss_rate * avg_loss  # per trade

    equity = pd.Series(equity_start + s.cumsum(), name='equity')
    run_max = equity.cummax()
    drawdown = equity - run_max
    max_dd = float(drawdown.min())  # negative

    std = float(s.std(ddof=1))
    sharpe_per_trade = (float(s.mean() - 0.04) / std * np.sqrt(n)) if std > 0 and n > 1 else np.nan

    stats = {
        'n_trades': n,
        'win_rate': win_rate,
        'avg_win': avg_win,
        'avg_loss': avg_loss,                 # negative number
        'payoff_ratio': payoff,               # |avg_win| / |avg_loss|
        'expectancy_per_trade': expectancy,   # same units as returns
        'gross_profit': gross_profit,
        'gross_loss': gross_loss,
        'profit_factor': profit_factor,
        'total_pnl': float(s.sum()),
        'median_pnl': float(s.median()),
        'max_drawdown': max_dd,               # same units as returns
        'sharpe_per_trade': sharpe_per_trade
    }

    if r_col and r_col in df.columns:
        r = df[r_col].astype(float)
        stats.update({
            'avg_R': float(r.mean()),
            'median_R': float(r.median()),
            'total_R': float(r.sum()),
            'win_rate_Rpos': float((r > 0).mean())
        })

    return stats #, equity, drawdown


stats = trade_stats(trades2, pnl_col='pnl_usd')
print(pd.Series(stats))




temp = trades.copy(deep=True)

if 'date' in temp.columns:
    temp['Date'] = pd.to_datetime(temp['date'], errors='coerce')
    df = temp.sort_values('date')

r = pd.to_numeric(df['pnl_usd'], errors='coerce').fillna(0)

# 1) Cumulative PnL (use for per-trade PnL/points)
df['cum_pnl'] = r.cumsum()


import matplotlib.pyplot as plt
plt.figure(figsize=(12,4))
plt.plot(df['Date'] if 'Date' in df.columns else df.index, df['cum_pnl'])
plt.title('Cumulative PnL')
plt.xlabel('Date' if 'Date' in df.columns else 'Trade #')
plt.ylabel('PnL')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

temp

n_trades                  1427.000000
win_rate                     0.250876
avg_win                    491.256983
avg_loss                  -153.344247
payoff_ratio                 3.203622
expectancy_per_trade         8.370708
gross_profit            175870.000000
gross_loss              163925.000000
profit_factor                1.072869
total_pnl                11945.000000
median_pnl                 -85.000000
max_drawdown             -6150.000000
sharpe_per_trade             0.884279
dtype: float64


NameError: name 'df' is not defined

In [35]:
import pandas as pd
import numpy as np

# ================== v3 PARAMETERS (tweak here) ==================
TICK_SIZE        = 0.25
POINT_VALUE      = 20.0      # NQ = $20/pt (use 2.0 for MNQ)
ATR_LEN          = 14
STOP_MULT        = 0.75      # stop = 0.75 * ATR
TARGET_MULT      = 2.0       # target = 2.0 * ATR
MOUTH_GAP_MULT   = 0.20      # mouth gaps must exceed 0.20 * ATR (was 0.15)
SLOPE_GATE_MULT  = 0.10      # |EMA200 slope over last 10 bars| > 0.15 * ATR
SLIPPAGE_TICKS   = 2.0       # next-bar-open entry slippage
SESSION_START    = "09:45:00"
SESSION_END      = "15:55:00"
COOLDOWN_AFTER_LOSS = 2      # skip next K signals after a stop
# ================================================================

# ---------- Indicators ----------
def smma(x: pd.Series, n: int) -> pd.Series:
    s = x.astype(float).copy()
    out = pd.Series(index=s.index, dtype=float)
    alpha = 1.0 / n
    prev = np.nan
    for i, v in enumerate(s.values):
        if i == 0 or np.isnan(prev):
            prev = v
        else:
            prev = prev + alpha * (v - prev)
        out.iat[i] = prev
    return out

def ema(x: pd.Series, n: int) -> pd.Series:
    return x.astype(float).ewm(span=n, adjust=False).mean()

def true_range(df: pd.DataFrame) -> pd.Series:
    pc = df["close"].shift(1)
    tr1 = df["high"] - df["low"]
    tr2 = (df["high"] - pc).abs()
    tr3 = (df["low"]  - pc).abs()
    return pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)

def atr(df: pd.DataFrame, n: int = 14) -> pd.Series:
    return smma(true_range(df), n)

# ---------- Prep ----------
def normalize_from_date_time(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out["datetime"] = pd.to_datetime(out["Date"].astype(str) + " " + out["Time"].astype(str))
    for c in ["open","high","low","close","volume"]:
        out[c] = pd.to_numeric(out[c], errors="coerce")
    out = out.dropna(subset=["open","high","low","close"]).sort_values("datetime").reset_index(drop=True)
    return out

# ---------- Backtest v3 ----------
def backtest_alligator_v3(
    raw_df: pd.DataFrame,
    tick_size=TICK_SIZE,
    point_value=POINT_VALUE,
    atr_len=ATR_LEN,
    stop_mult=STOP_MULT,
    target_mult=TARGET_MULT,
    mouth_gap_mult=MOUTH_GAP_MULT,
    slope_gate_mult=SLOPE_GATE_MULT,
    slippage_ticks=SLIPPAGE_TICKS,
    session_start=SESSION_START,
    session_end=SESSION_END,
    cooldown_after_loss=COOLDOWN_AFTER_LOSS
):
    df = normalize_from_date_time(raw_df)

    # Indicators
    df["ema200"] = ema(df["close"], 200)
    df["jaw"]    = smma(df["close"], 13)   # unshifted for signals
    df["teeth"]  = smma(df["close"], 8)
    df["lips"]   = smma(df["close"], 5)
    df["atr"]    = atr(df, atr_len)

    # EMA slope over last 10 bars
    df["ema_slope"] = df["ema200"] - df["ema200"].shift(10)

    # Intraday window
    t = df["datetime"].dt.time
    df = df[(t >= pd.to_datetime(session_start).time()) & (t <= pd.to_datetime(session_end).time())].copy()
    df.reset_index(drop=True, inplace=True)
    if len(df) < 50:
        return pd.DataFrame(), pd.Series({"n_trades": 0})

    trades = []
    in_pos = False
    side = 0
    entry_px = np.nan
    entry_time = None
    stop_px = np.nan
    tgt_px = np.nan
    cooldown = 0

    for i in range(1, len(df)-1):
        prev = df.iloc[i-1]
        row  = df.iloc[i]
        nxt  = df.iloc[i+1]

        # Manage open position
        if in_pos:
            exit_px = None
            reason = None
            if side == 1:
                if row["low"] <= stop_px:
                    exit_px, reason = stop_px, "stop"
                elif row["high"] >= tgt_px:
                    exit_px, reason = tgt_px, "target"
            else:
                if row["high"] >= stop_px:
                    exit_px, reason = stop_px, "stop"
                elif row["low"] <= tgt_px:
                    exit_px, reason = tgt_px, "target"

            if exit_px is not None:
                exit_px = round(exit_px / tick_size) * tick_size
                pnl_pts = (exit_px - entry_px) * side
                pnl_usd = pnl_pts * point_value
                trades.append({
                    "entry_time": entry_time, "entry_px": entry_px, "side": side,
                    "exit_time": row["datetime"], "exit_px": exit_px, "reason": reason,
                    "pnl_pts": pnl_pts, "pnl_usd": pnl_usd
                })
                # Cooldown on loss
                if reason == "stop":
                    cooldown = cooldown_after_loss
                in_pos = False
                side = 0
                continue

        # Entries only if flat
        if not in_pos:
            # Cooldown management
            if cooldown > 0:
                cooldown -= 1
                continue

            if row["atr"] <= 0 or pd.isna(row["atr"]):
                continue

            bias_long  = (row["close"] > row["ema200"]) and (row["ema_slope"] > 0)
            bias_short = (row["close"] < row["ema200"]) and (row["ema_slope"] < 0)

            # Mouth open with ATR gap requirement
            mouth_up = (row["lips"] > row["teeth"]) and (row["teeth"] > row["jaw"]) \
                       and ((row["lips"] - row["teeth"]) > mouth_gap_mult*row["atr"]) \
                       and ((row["teeth"] - row["jaw"])  > mouth_gap_mult*row["atr"])

            mouth_dn = (row["lips"] < row["teeth"]) and (row["teeth"] < row["jaw"]) \
                       and ((row["teeth"] - row["lips"]) > mouth_gap_mult*row["atr"]) \
                       and ((row["jaw"]  - row["teeth"]) > mouth_gap_mult*row["atr"])

            crossed_up = (prev["close"] <= prev["teeth"]) and (row["close"] > row["teeth"])
            crossed_dn = (prev["close"] >= prev["teeth"]) and (row["close"] < row["teeth"])

            # Slope magnitude gate
            slope_gate = abs(row["ema_slope"]) > slope_gate_mult * row["atr"]

            # Long
            if bias_long and mouth_up and crossed_up and slope_gate:
                raw = float(nxt["open"])
                entry_px = round((raw + slippage_ticks * tick_size) / tick_size) * tick_size
                side = 1
                entry_time = nxt["datetime"]
                stop_px = entry_px - stop_mult * row["atr"]
                tgt_px  = entry_px + target_mult * row["atr"]
                in_pos = True
                continue

            # Short
            if bias_short and mouth_dn and crossed_dn and slope_gate:
                raw = float(nxt["open"])
                entry_px = round((raw - slippage_ticks * tick_size) / tick_size) * tick_size
                side = -1
                entry_time = nxt["datetime"]
                stop_px = entry_px + stop_mult * row["atr"]
                tgt_px  = entry_px - target_mult * row["atr"]
                in_pos = True
                continue

    # Results
    trades_df = pd.DataFrame(trades)
    if trades_df.empty:
        summary = pd.Series({
            "n_trades": 0, "win_rate": np.nan,
            "total_pnl_pts": 0.0, "total_pnl_usd": 0.0,
            "avg_pnl_pts": np.nan, "max_drawdown_usd": 0.0
        })
        return trades_df, summary

    trades_df["cum_pnl_usd"] = trades_df["pnl_usd"].cumsum()
    roll_max = trades_df["cum_pnl_usd"].cummax()
    dd = trades_df["cum_pnl_usd"] - roll_max
    summary = pd.Series({
        "n_trades": int(len(trades_df)),
        "win_rate": float((trades_df["pnl_usd"] > 0).mean()),
        "total_pnl_pts": float(trades_df["pnl_pts"].sum()),
        "total_pnl_usd": float(trades_df["pnl_usd"].sum()),
        "avg_pnl_pts": float(trades_df["pnl_pts"].mean()),
        "max_drawdown_usd": float(dd.min())
    })
    return trades_df, summary

# -------- RUN v3 on your NQ_Data --------
trades3, summary3 = backtest_alligator_v3(NQ_Data)
print("=== SUMMARY v3 (ATR + slope gate + cooldown) ===")
print(summary3)
print("\nSample trades:")
trades3


=== SUMMARY v3 (ATR + slope gate + cooldown) ===
n_trades             874.000000
win_rate               0.242563
total_pnl_pts         55.000000
total_pnl_usd       1100.000000
avg_pnl_pts            0.062929
max_drawdown_usd   -7810.000000
dtype: float64

Sample trades:


,entry_time,entry_px,side,exit_time,exit_px,reason,pnl_pts,pnl_usd,cum_pnl_usd
0,2024-11-01 11:03:00,20115.25,1,2024-11-01 11:03:00,20112.25,stop,-3.00,-60.0,-60.0
1,2024-11-01 11:29:00,20112.25,1,2024-11-01 11:35:00,20121.00,target,8.75,175.0,115.0
2,2024-11-01 13:11:00,20094.75,-1,2024-11-01 13:12:00,20101.75,stop,-7.00,-140.0,-25.0
3,2024-11-01 15:23:00,20255.50,1,2024-11-01 15:27:00,20285.25,target,29.75,595.0,570.0
4,2024-11-04 14:21:00,20118.00,-1,2024-11-04 14:25:00,20104.25,target,13.75,275.0,845.0
...,...,...,...,...,...,...,...,...,...
869,2025-10-30 11:33:00,26271.75,1,2025-10-30 11:34:00,26266.75,stop,-5.00,-100.0,910.0
870,2025-10-30 13:08:00,26100.00,-1,2025-10-30 13:08:00,26110.25,stop,-10.25,-205.0,705.0
871,2025-10-31 10:26:00,26166.50,-1,2025-10-31 10:29:00,26170.25,stop,-3.75,-75.0,630.0
872,2025-10-31 14:08:00,26097.00,-1,2025-10-31 14:08:00,26112.50,stop,-15.50,-310.0,320.0


In [28]:
def sweep_v3(NQ_Data):
    i = 0
    rows = []
    for stop_mult in [0.75]:
        for target_mult in [2.0, 2.5, 3.0]:
            for mouth_gap in [0.20, 0.25]:
                for slope_mult in [0.10, 0.15]:
                    for cooldown in [3]:
                        print(i)
                        i= i+1
                        tr, sm = backtest_alligator_v3(
                            NQ_Data,
                            stop_mult=stop_mult,
                            target_mult=target_mult,
                            mouth_gap_mult=mouth_gap,
                            slope_gate_mult=slope_mult,
                            slippage_ticks=2.0,
                            cooldown_after_loss=cooldown
                        )
                        rows.append({
                            "stop_mult": stop_mult,
                            "target_mult": target_mult,
                            "mouth_gap": mouth_gap,
                            "slope_mult": slope_mult,
                            "cooldown": cooldown,
                            "n_trades": sm["n_trades"],
                            "win_rate": sm["win_rate"],
                            "avg_pts": sm["avg_pnl_pts"],
                            "total_usd": sm["total_pnl_usd"],
                            "max_dd": sm["max_drawdown_usd"]
                        })
    return pd.DataFrame(rows).sort_values(["total_usd","max_dd"], ascending=[False, True])

grid = sweep_v3(NQ_Data)
print(grid.head(15))


0
1
2
3
4
5
6
7
8
9
10
11
    stop_mult  target_mult  mouth_gap  slope_mult  cooldown  n_trades  \
0        0.75          2.0       0.20        0.10         3     850.0   
8        0.75          3.0       0.20        0.10         3     819.0   
4        0.75          2.5       0.20        0.10         3     838.0   
9        0.75          3.0       0.20        0.15         3     761.0   
1        0.75          2.0       0.20        0.15         3     791.0   
5        0.75          2.5       0.20        0.15         3     779.0   
10       0.75          3.0       0.25        0.10         3     540.0   
6        0.75          2.5       0.25        0.10         3     545.0   
2        0.75          2.0       0.25        0.10         3     547.0   
11       0.75          3.0       0.25        0.15         3     503.0   
7        0.75          2.5       0.25        0.15         3     508.0   
3        0.75          2.0       0.25        0.15         3     510.0   

    win_rate   avg_pts  

In [29]:
grid

,stop_mult,target_mult,mouth_gap,slope_mult,cooldown,n_trades,win_rate,avg_pts,total_usd,max_dd
0,0.75,2.0,0.20,0.10,3,850.0,0.265882,0.667941,11355.0,-4535.0
8,0.75,3.0,0.20,0.10,3,819.0,0.190476,0.653236,10700.0,-5845.0
4,0.75,2.5,0.20,0.10,3,838.0,0.224344,0.544451,9125.0,-5145.0
9,0.75,3.0,0.20,0.15,3,761.0,0.189225,0.521025,7930.0,-6100.0
1,0.75,2.0,0.20,0.15,3,791.0,0.261694,0.447851,7085.0,-5295.0
5,0.75,2.5,0.20,0.15,3,779.0,0.220796,0.374519,5835.0,-6285.0
10,0.75,3.0,0.25,0.10,3,540.0,0.194444,0.359259,3880.0,-4810.0
6,0.75,2.5,0.25,0.10,3,545.0,0.233028,0.316055,3445.0,-4300.0
2,0.75,2.0,0.25,0.10,3,547.0,0.265082,0.262797,2875.0,-4905.0
11,0.75,3.0,0.25,0.15,3,503.0,0.192843,0.195328,1965.0,-5410.0


In [27]:
grid

,stop_mult,target_mult,mouth_gap,slope_mult,cooldown,n_trades,win_rate,avg_pts,total_usd,max_dd
18,0.75,2.0,0.20,0.10,3,850.0,0.265882,0.667941,11355.0,-4535.0
19,0.75,2.0,0.20,0.15,3,791.0,0.261694,0.447851,7085.0,-5295.0
9,0.75,1.5,0.20,0.10,3,863.0,0.308227,0.194959,3365.0,-5365.0
21,0.75,2.0,0.25,0.10,3,547.0,0.265082,0.262797,2875.0,-4905.0
10,0.75,1.5,0.20,0.15,3,803.0,0.306351,0.069427,1115.0,-6055.0
20,0.75,2.0,0.20,0.20,3,735.0,0.254422,0.055442,815.0,-7500.0
12,0.75,1.5,0.25,0.10,3,551.0,0.313975,0.028131,310.0,-4540.0
25,0.75,2.0,0.30,0.15,3,295.0,0.264407,0.004237,25.0,-5685.0
24,0.75,2.0,0.30,0.10,3,312.0,0.262821,-0.002404,-15.0,-5455.0
22,0.75,2.0,0.25,0.15,3,510.0,0.260784,-0.006373,-65.0,-5850.0
